In [466]:
#ignore, used to look up OG contents of input file
import os
import pandas as pd
import numpy as np
from scipy.stats import zscore

path = './data/'
filename_read = os.path.join(path,'heart.csv')
df = pd.read_csv(filename_read)


In [467]:
#Useful functions
import os
import pandas as pd

path = "./data/"

filename_read = os.path.join(path, "heart.csv")
df = pd.read_csv(filename_read)

#Randomize the data
np.random.seed(50)
df = df.reindex(np.random.permutation(df.index))
df.reset_index(inplace=True, drop=True)


headers = list(df.columns.values)
fields = []

for field in headers:
    fields.append({
        'data Val' : field,
        'mean' : df[field].mean(),
        'var' : df[field].var(),
        'sdev' : df[field].std()
    })
    
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

    


In [545]:
#Classification Neural Network

import tensorflow as tf 
import pandas as pd 
import numpy as np 
import requests 
import io 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from keras.models import Sequential 
from keras.layers.core import Dense, Activation 
from keras.callbacks import EarlyStopping

path = "./data/"

filename_read = os.path.join(path, "heart.csv") 
df = pd.read_csv(filename_read)

#one-hot coding for catagory classification data 
one_hot = pd.get_dummies(df['sex']) 
df = df.drop('sex', axis = 1) 
df = df.join(one_hot) 
df.rename(columns = { 0: "female",
                     1: "male"}, 
          inplace = True) 
one_hot = pd.get_dummies(df['fbs']) 
df = df.drop('fbs', axis = 1) 
df = df.join(one_hot)
df.rename(columns = { 0: "fbs-no",
                     1: "fbs-yes"}, 
          inplace = True) 

one_hot = pd.get_dummies(df['exang'])
df = df.drop('exang', axis = 1) 
df = df.join(one_hot)
df.rename(columns = { 0: "exang-no",
                     1: "exang-yes"},
          inplace = True)

#Normalize the real number data
df['age'] = zscore(df['age']) 
df['cp'] = zscore(df['cp']) 
df['trestbps'] = zscore(df['trestbps']) 
df['chol'] = zscore(df['chol']) 
df['restecg'] = zscore(df['restecg']) 
df['age'] = zscore(df['age']) 
df['thalach'] = zscore(df['thalach']) 
df['oldpeak'] = zscore(df['oldpeak']) 
df['slope'] = zscore(df['slope']) 
df['ca'] = zscore(df['ca']) 
df['thal'] = zscore(df['thal'])

df = df.reindex(np.random.permutation(df.index))#shuffle the data 
df.reset_index(inplace=True, drop=True)

#Pre-segment the data
mask = np.random.rand(len(df)) < 0.7
trainDF = pd.DataFrame(df[mask])
validationDF = pd.DataFrame(df[~mask])


x,y = to_xy(trainDF, "target")
x1,y1 = to_xy(validationDF, "target")

#split into train/test 
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.30, random_state=42)#Train on 70% Tes on 30

model = Sequential()
model.add(Dense(2, input_dim=x.shape[1], activation='relu'))#Input layer
model.add(Dense(1,activation='relu')) #Hidden Layer
model.add(Dense(y.shape[1],activation='softmax')) #Output Layer
model.compile(loss='categorical_crossentropy', optimizer='adam') 
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,patience=5, verbose=1, mode='auto')#Stop, when improvments arn't made for 5 turns              

model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor], verbose=2,epochs=500)

Train on 205 samples, validate on 62 samples
Epoch 1/500
 - 3s - loss: 0.6924 - val_loss: 0.6987
Epoch 2/500
 - 0s - loss: 0.6879 - val_loss: 0.6935
Epoch 3/500
 - 0s - loss: 0.6833 - val_loss: 0.6891
Epoch 4/500
 - 0s - loss: 0.6795 - val_loss: 0.6847
Epoch 5/500
 - 0s - loss: 0.6757 - val_loss: 0.6803
Epoch 6/500
 - 0s - loss: 0.6719 - val_loss: 0.6761
Epoch 7/500
 - 0s - loss: 0.6678 - val_loss: 0.6720
Epoch 8/500
 - 0s - loss: 0.6639 - val_loss: 0.6679
Epoch 9/500
 - 0s - loss: 0.6597 - val_loss: 0.6640
Epoch 10/500
 - 0s - loss: 0.6559 - val_loss: 0.6604
Epoch 11/500
 - 0s - loss: 0.6523 - val_loss: 0.6568
Epoch 12/500
 - 0s - loss: 0.6487 - val_loss: 0.6531
Epoch 13/500
 - 0s - loss: 0.6446 - val_loss: 0.6496
Epoch 14/500
 - 0s - loss: 0.6408 - val_loss: 0.6462
Epoch 15/500
 - 0s - loss: 0.6371 - val_loss: 0.6428
Epoch 16/500
 - 0s - loss: 0.6326 - val_loss: 0.6393
Epoch 17/500
 - 0s - loss: 0.6285 - val_loss: 0.6359
Epoch 18/500
 - 0s - loss: 0.6240 - val_loss: 0.6325
Epoch 19/5

Epoch 154/500
 - 0s - loss: 0.3861 - val_loss: 0.3702
Epoch 155/500
 - 0s - loss: 0.3853 - val_loss: 0.3690
Epoch 156/500
 - 0s - loss: 0.3848 - val_loss: 0.3677
Epoch 157/500
 - 0s - loss: 0.3839 - val_loss: 0.3666
Epoch 158/500
 - 0s - loss: 0.3834 - val_loss: 0.3655
Epoch 159/500
 - 0s - loss: 0.3827 - val_loss: 0.3642
Epoch 160/500
 - 0s - loss: 0.3821 - val_loss: 0.3635
Epoch 161/500
 - 0s - loss: 0.3813 - val_loss: 0.3626
Epoch 162/500
 - 0s - loss: 0.3809 - val_loss: 0.3616
Epoch 163/500
 - 0s - loss: 0.3804 - val_loss: 0.3610
Epoch 164/500
 - 0s - loss: 0.3798 - val_loss: 0.3604
Epoch 165/500
 - 0s - loss: 0.3793 - val_loss: 0.3602
Epoch 166/500
 - 0s - loss: 0.3787 - val_loss: 0.3599
Epoch 167/500
 - 0s - loss: 0.3784 - val_loss: 0.3590
Epoch 168/500
 - 0s - loss: 0.3778 - val_loss: 0.3584
Epoch 169/500
 - 0s - loss: 0.3773 - val_loss: 0.3578
Epoch 170/500
 - 0s - loss: 0.3768 - val_loss: 0.3568
Epoch 171/500
 - 0s - loss: 0.3763 - val_loss: 0.3558
Epoch 172/500
 - 0s - loss: 

In [546]:
from sklearn import metrics
import tensorflow as tf

#Create Predictions for each person in the (Prediction : Level Of Confidence)
pred = model.predict(x_test)
print(pred)

[[0.3022 0.6978]
 [0.2381 0.7619]
 [0.1551 0.8449]
 [0.1647 0.8353]
 [0.1551 0.8449]
 [0.6751 0.3249]
 [0.1551 0.8449]
 [0.1551 0.8449]
 [0.1551 0.8449]
 [0.1551 0.8449]
 [0.2016 0.7984]
 [0.1551 0.8449]
 [0.1551 0.8449]
 [0.506  0.494 ]
 [0.9384 0.0616]
 [0.9594 0.0406]
 [0.8057 0.1943]
 [0.1551 0.8449]
 [0.9976 0.0024]
 [0.9831 0.0169]
 [0.9916 0.0084]
 [0.7259 0.2741]
 [0.9682 0.0318]
 [0.1551 0.8449]
 [0.1551 0.8449]
 [0.1923 0.8077]
 [0.1551 0.8449]
 [0.3196 0.6804]
 [0.9225 0.0775]
 [0.1551 0.8449]
 [0.8902 0.1098]
 [0.1551 0.8449]
 [0.9612 0.0388]
 [0.548  0.452 ]
 [0.1551 0.8449]
 [0.226  0.7739]
 [0.1551 0.8449]
 [0.993  0.007 ]
 [0.1551 0.8449]
 [0.815  0.185 ]
 [0.1551 0.8449]
 [0.9324 0.0676]
 [0.1622 0.8378]
 [0.9977 0.0023]
 [0.1551 0.8449]
 [0.3654 0.6346]
 [0.6695 0.3305]
 [0.1574 0.8426]
 [0.1551 0.8449]
 [0.1574 0.8426]
 [0.2159 0.7841]
 [0.99   0.01  ]
 [0.172  0.828 ]
 [0.1551 0.8449]
 [0.1551 0.8449]
 [0.9884 0.0116]
 [0.1551 0.8449]
 [0.279  0.721 ]
 [0.844  0.156

In [547]:
#Displays the raw probability of chosen class (prediction for test set)
pred = np.argmax(pred,axis=1)
print(pred)

[1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1
 0 1 0 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1]


In [548]:
#Calculates the percent accuacy of correct classification
y_compare = np.argmax(y_test, axis=1)
score = metrics.accuracy_score(y_compare, pred)
print("Accuracy Score: {}".format(score))

Accuracy Score: 0.9032258064516129


In [549]:
y_compare == pred

array([ True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True])

In [550]:
#Calculate log loss (Inaccuracy of predictions)
from IPython.display import display

# Dont display numpy in SI notation
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

# Generate predictions
pred = model.predict(x_test)

print("Numpy array of predictions")
display(pred[0]*100)

print("As percent probability")
display(pred[0:5]*100)

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))

Numpy array of predictions


array([30.2182, 69.7818], dtype=float32)

As percent probability


array([[30.2182, 69.7818],
       [23.815 , 76.185 ],
       [15.5051, 84.4949],
       [16.4655, 83.5346],
       [15.5051, 84.4949]], dtype=float32)

Log loss score: 0.30781720820513947
